# How To Use NLP

## 2. Word2Vec으로 문장을 벡터로 변환하기

$ pip install gensim

### 2.1 Genism으로 토지 읽어들이고 Word2Vec로 embedding하기

In [1]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter

#### utp-16인코딩으로 파일을 열고 글자를 출력하기

In [3]:
fp = codecs.open('../data/BEXX0003.txt','r', encoding='utf-16')
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body > text")
text = body.getText()

#### 텍스트를 한 줄씩 처리하기

In [4]:
twitter = Twitter()

In [5]:
results = []
#split by lines
lines = text.split("\r\n")

for line in lines:
    # 형태소 분석하기 - 단어의 기본형 사용
    poslist = twitter.pos(line, norm=True, stem=True)
    r = []
    for word in poslist:
        # 어미/조사/구두점 등은 대상에서 제외 
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            r.append(word[0])
    readlines = (" ".join(r)).strip()
    results.append(readlines)
    print(readlines[:50])
    print(len(readlines))


제 1 편 어둠 발 소리 서다 序 1897 년 한가위 까치 들 울타리 안 감나무 와 아침 
165704


#### 파일로 출력하기

In [6]:
origins = '../data/toji.origin'
with open(origins, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

#### Word2Vec 모델 만들기

word2vec 파라메터 설명
config = {  
    'min_count': 5,  # 등장 횟수가 5 이하인 단어는 무시  
    'window' : skip gram에서, 주변에 몇개의 단어까지 볼 것인가. 보통 5단어 정도로 한다  
    'hs' : hirachical softmax, negative sampling flag, 모델 복잡도를 개선하기 위해 사용함, 보통 negative sampling의 성능이 더 좋음  
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding, 보통 300차원 정도로 한다.  
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다. 보통 skip-gram을 많이 사용  
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수  
    'iter': 5,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수  
    'workers': multiprocessing.cpu_count(),  
}  

In [8]:
from gensim.models import word2vec

data = word2vec.LineSentence(origins)
model = word2vec.Word2Vec(data, 
    size=200, window=10, hs=1, min_count=2, sg=1)
model.save("../data/toji.bin")
print("ok")

ok


### 2.2 사용한 language model을 바탕으로 유사단어 검출하기

In [9]:
model = word2vec.Word2Vec.load("../data/toji.bin")

In [10]:
model.wv.most_similar(positive =["땅"])

[('장차', 0.8618342876434326),
 ('벼슬길', 0.846946120262146),
 ('전답', 0.8333130478858948),
 ('쉬다', 0.8295556902885437),
 ('조상', 0.8258488178253174),
 ('망치다', 0.8249940872192383),
 ('박서방', 0.8247364163398743),
 ('게다가', 0.8245334029197693),
 ('경신', 0.8241356015205383),
 ('사시', 0.822289228439331)]

In [11]:
model.wv.most_similar(positive=["땅","어머니"])

[('이쯤', 0.9076478481292725),
 ('생생', 0.9064553380012512),
 ('손주', 0.9051131010055542),
 ('골골', 0.9043962359428406),
 ('소요', 0.9020668864250183),
 ('벼슬길', 0.9011671543121338),
 ('사태', 0.9004140496253967),
 ('심청', 0.8998978734016418),
 ('판술', 0.8985966444015503),
 ('경신', 0.8981589078903198)]

## Summerize : Word2Vec을 이용하여 특정 단어의 유의어, 반의어 등 추출 가능  
**Word2Vec으로 단어를 linear expression이 가능하다.**